# Import Useful Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from enterprise.pulsar import Pulsar
from enterprise_extensions import blocks
import pickle
import NimaClass as nc
from astropy.coordinates import SkyCoord
from astropy import units as u
from enterprise.signals import gp_signals, signal_base
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
from enterprise_extensions import sampler as samp
import sim
import corner
%load_ext autoreload
%autoreload 2

# Plotting Settings

Note: If you do not like a dark_background for your plots, do not run `plt.style.use('dark_background')`

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('dark_background')
hist_settings = dict(
    bins = 40,
    histtype = 'step',
    lw = 3,
    density = True
)

# <center><span style="color:#7cfc00">Introduction

The goal of detection is to assess the statistical significance of various types of <span style="color:red">red noise processes</span>. To achieve this goal, we need to make sure we understand a few topics defined within the context of pulsar timing experiments.

## <span style="color:#7cfc00">Topic 1: What is a noise process?

A noise process is a series of random numbers which are completely independent of each other. This means you cannot predict the future given the past, in complete contrast to deterministic processes. 

We deal with two types of noise processes:
1. White (Gaussian) Noise: random numbers drawn from a Gaussian distribution with the property that their spectral power is constant across all frequencies.
2. Red Noise: a linear combination of white noise processes filtered in a way that its spectral power decreases with increasing frequencies.

In case you are not familiar with spectral power, you can think of it as a measure of the strength of the signal that the series of random numbers represent. This power is defined in the frequency domain. It is an extremely important quantity!

### <span style="color:#7cfc00">White Noise

$\Large w_{\underbrace{I}_{\text{pulsar index}}}$

In [ ]:
draws = int(1e6)
wn = np.random.normal(loc = 0, scale = .2, size = draws)
t = np.linspace(0, 5400, draws)

idxs = np.random.randint(0, draws, 500)
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (6*2, 6))
ax[0].hist(wn, **hist_settings)
ax[0].set_title('White Noise -- Probability Distribution')
ax[1].scatter(t[idxs], wn[idxs])
ax[1].set_title('White Noise -- The Process')
ax[0].set_xlabel('White Noise values')
ax[1].set_xlabel('Time')
ax[0].set_ylabel('Density')
ax[1].set_ylabel('White Noise values')
plt.tight_layout()
plt.show()

### <span style="color:#7cfc00">Red Noise
$\Large r_I(f) = c_I(f)w_I(f)$

$\Large {\underbrace{c_I}_{\text{filtering function that sets the spectral power}}}$

In [ ]:
rn = np.load('../Data/red_noise_example.npz')['rn']
t = np.linspace(0, 100, len(rn))

fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (6*2, 6))
ax[0].hist(rn, **hist_settings)
ax[0].set_title('Red Noise -- Probability Distribution')
ax[1].scatter(t, rn)
ax[1].set_title('Red Noise -- The Process')
ax[0].set_xlabel('Red Noise values')
ax[1].set_xlabel('Time')
ax[0].set_ylabel('Density')
ax[1].set_xlabel('Time')
plt.tight_layout()
plt.show()

### <span style="color:#7cfc00">Note:

1. When dealing with noise, the concept of <span style="color:red">realization</span> becomes important. One can construct infinite number of unique-looking signals in the time-domain all of which possessing the same spectral properties (e.g., identical spectral power). A realization of a noise process is one out of infinite number of such possibilities. When doing computation, changing the seed of a random number generator changes the realization of the noise process made out of the random numbers.

2. The mean of a red noise process is not zero, unlike a white noise process, if you take the mean over a single realization. In theory, a true red noise process is a zero-mean process if the mean is taken over many realizations. Make sure you understand the difference between them!

## <span style="color:#7cfc00">Topic 2: What is a common red noise process?
$\Large r_I(f) = c(f)w_I(f)$

Definition: If different realizations of the same red noise process exist between different pulsars' timing residuals, the process is called a common red noise process. 

Question: How to go about checking if multiple realizations of the same red noise process exist in different pulsars? What statistical quantity could reveal such information?

Note: note the difference between how $r_I$ is defined as a common red noise versus as a red noise.

## <span style="color:#7cfc00"> Topic 3: What is a common correlated red noise process?

$\Large r_I(f) = c(f)\sum_{J}\Gamma_{IJ}w_J(f)$

$\Large {\underbrace{\Gamma_{IJ}}_{\text{a non-diagonal matrix}}}$

Definition: if a pair of two different realizations of the same red noise process are correlated, the red noise process between the pair is called a common correlated red noise process. 

If the non-diagonal elements of the matrix $\large \Gamma$ depend on pulsar pairs' angular separation, the correlation is called spatial correlation. This type of correlation is searched for in PTA GWB analyses.

## <span style="color:#7cfc00">Topic 4: What is an intrinsic red noise process?

Definition: a red noise process that is not common between pulsars

## <span style="color:#7cfc00">Topic 5: What is spectral power?

Spectral power, power spectral density, and PSD all refer to the same quantity. As described earlier, PSD is a measure of a noise process's strength defined in the frequency domain. Mathematically,

$\Large \langle r_I(f)r_I^*(f')\rangle  = \delta(f-f') \times \text{PSD} $,

where $\delta$ is the kronecker delta function.

You can model a red noise's PSD in any shape you want as long as the value of PSD decreases with increasing frequency. The two simplest and the most used models are 
1. Powerlaw model

2. Free spectral model
The choice for the way you model PSD of a red noise process could be astrophysically motivated. You will hear about this in future tutorials!

### <span style="color:#7cfc00">Powerlaw Model of PSD

$\Large P(f_k) = \frac{A^2}{12 \pi^2 f_k^3} \left( \frac{f_k}{f_{\text{ref}}}\right)^{3-\gamma},$

$k$ is the frequency index, $A$ is the amplitude, $f_\text{ref}$ is the reference frequency (typically 1/1yr), and $\gamma$ is the spectral index which must be a positive number.

### <span style="color:#7cfc00">Free-spectral Model of PSD

$\Large P(f_k) = T_{\text{obs}}\rho_k^2$

$T_{\text{obs}}$ is the observational time and $\rho_k^2$ is the value of PSD at frequency-bin $k$. Notice the difference between a powerlaw model and a free-spectral model. In a free-spectral model, each frequency-bin is allowed to have its own PSD independent of other bins.

# <center><span style="color:#7cfc00">ENTERPRISE: a PTA Data Analysis Tool

![title](../Image/ENTERPRISE.png)


## <span style="color:#7cfc00"> Input

In [ ]:
##The ENTERPRISE python object holding information from par & tim files
psrs = []

### <span style="color:#7cfc00"> In case you have par & tim files:

In [ ]:
# # Path to tim file(s)
# timfiles = sorted(glob.glob('...' + '/*.tim'))
# # Path to par file(s)
# parfiles = sorted(glob.glob('...'+ '/*.par'))
# for p, t in zip(parfiles, timfiles):
#     psrs.append(Pulsar(p, t, ephem = None , clk=None))

### <span style="color:#7cfc00"> In case you have a pickle file of the psrs object

In [ ]:
with open('../Data/Pickle/0.pkl', 'rb') as fin:
    psrs = pickle.load(fin)
psrs

## <span style="color:#7cfc00"> Exploring the `psrs` object 

In [ ]:
## all the atributes of the psrs object
dir(psrs[0])

In [ ]:
## number of pulsars
Npulsars = len(psrs)
Npulsars

In [ ]:
## name of the pulsars
psrlist = [psr.name for psr in psrs]
psrlist

In [ ]:
## toas and residuals
for psr in psrs[:5]:
    plt.plot(psr.toas/(86400), psr.residuals)
    plt.title(psr.name)
    plt.xlabel('TOAs [MJD]')
    plt.ylabel('Timing Residuals (s)')
    plt.tight_layout()
    plt.show()

In [ ]:
## pulsars' sky location
dec = np.array([psr._decj for psr in psrs])
ra = np.array([psr._raj for psr in psrs])

In [ ]:
sc = SkyCoord(ra = ra , dec = dec, unit = 'rad', frame='icrs')
plt.figure(figsize=(12, 6))
plt.subplot(projection="aitoff")

c = SkyCoord(ra*180/np.pi, dec*180/np.pi, unit = 'deg', frame='icrs')
ra_rad = c.ra.wrap_at(180 * u.deg).radian
dec_rad = c.dec.radian
plt.scatter(ra_rad  , dec_rad,marker=(5, 2),color = 'r',label = 'Pulsars')

plt.xticks(ticks=np.radians([-150, -120, -90, -60, -30, 0, \
                                30, 60, 90, 120, 150]),
            labels=['10h', '8h', '6h', '4h', '2h', '0h', \
                    '22h', '20h', '18h', '16h', '14h'])

plt.xlabel('Right Ascension in hours')
plt.ylabel('Declination in deg.')
plt.grid(True)
plt.legend(loc = 'upper right')
plt.tight_layout()
plt.show()

In [ ]:
## Pulsar distances and their error (mean, std)
[psr.pdist for psr in psrs]

In [ ]:
## TOA errors in seconds
[psr.toaerrs for psr in psrs]

In [ ]:
## The observing baseline for each pulsar (in years)
tspans = np.array([(psr.toas.max() - psr.toas.min())/(86400 * 365.25) for psr in psrs])
tspans

## <span style="color:#7cfc00"> Noise Modeling Using ENTERPRISE

$\Large R(t) = M \epsilon + Fa + \text{WN} $

### <span style="color:#7cfc00"> Timing Model

When we construct a timing model for a pulsar, we have uncertainties about our model. This uncertainty $\epsilon$ needs to be taken into consideration in any noise modeling that we do.
The matrix $M$ is the design-matrix containing the bases of the timing model parameters.

In [ ]:
tm = gp_signals.MarginalizingTimingModel(use_svd=True)

### <span style="color:#7cfc00"> White Noise

In [ ]:
## Do you need to vary the parameters that describe white noise (EFAC, EQUAD, and ECORR are the params)?
vary = False
if not vary:
    noise_dict = {}
    for pname in psrlist:
        noise_dict.update({pname + '_efac': 1.0})
        noise_dict.update({pname + '_log10_t2equad': -np.inf})
## Do you need to include ECORR noise (do not worry if you do not know what ECORR is!)?
inc_ecorr = False
## Do you want to use different backends for the telescopes?
select = 'none'

wn = blocks.white_noise_block(vary = vary, inc_ecorr = inc_ecorr, select = select)

### <span style="color:#7cfc00"> Intrinsic Red Noise

In [ ]:
## How do you want to model the spectral power of your red noise? 
psd = 'powerlaw'
## What do you know about the prior for the parameters that describe the red noise?
prior = 'log-uniform'
## What frequencies do you want to consider in your model? It must be in seconds.
number_of_bins = 30
observing_time_to_use_in_freq_calculation = tspans.max() * 86400 * 365.25

rn = blocks.red_noise_block(psd=psd, prior=prior, Tspan=observing_time_to_use_in_freq_calculation,
                            components=number_of_bins)

### <span style="color:#7cfc00">Common Red Noise

In [ ]:
## How do you want to model the spectral power of your red noise? 
psd = 'powerlaw'
## What do you know about the prior for the parameters that describe the red noise?
prior = 'log-uniform'
## What frequencies do you want to consider in your model? It must be in seconds.
number_of_bins = 5
observing_time_to_use_in_freq_calculation = tspans.max() * 86400 * 365.25

crn = blocks.common_red_noise_block(psd=psd, prior=prior, 
                                    Tspan=observing_time_to_use_in_freq_calculation, 
                                    components=number_of_bins,
                                    name = 'crn')


### <span style="color:#7cfc00">Common Correlated Red Noise

In [ ]:
## How do you want to model the spectral power of your red noise? 
psd = 'powerlaw'
## What do you know about the prior for the parameters that describe the red noise?
prior = 'log-uniform'
## What frequencies do you want to consider in your model? It must be in seconds.
number_of_bins = 5
observing_time_to_use_in_freq_calculation = tspans.max() * 86400 * 365.25
## What functional shape do you want the correlations to follow?
corr = 'hd'

ccrn = blocks.common_red_noise_block(psd=psd, prior=prior, 
                                    Tspan=observing_time_to_use_in_freq_calculation, 
                                    components=number_of_bins, orf = corr,
                                    name = 'gw')


### <span style="color:#7cfc00">Build the Model

In [ ]:
#signal = tm + wn + rn
#signal = tm + wn + crn
#signal = tm + wn + ccrn
signal = tm + wn + rn + crn

In [ ]:
## How many pulsars do you want to consider in your model?
psrs_to_choose = psrs[:]
## Build the model
pta = signal_base.PTA([signal(p) for p in psrs_to_choose], lnlikelihood = signal_base.LogLikelihoodDenseCholesky)
## You need to fix the white noise parameters if you chose not to vary them when defining `wn`
if not vary:
    pta.set_default_params(noise_dict)

In [ ]:
pta.params

### <span style="color:#7cfc00">Calculate the log-liklihood of the model 

In [ ]:
x0 = np.hstack([p.sample() for p in pta.params])
pta.get_lnlikelihood(x0)

# <span style="color:#7cfc00"> MCMC Sampler

In [ ]:
## Output directory
outdir = '../Chain/Fall/MCMC'

## Jump Proposals, groups, and MCMC Tunings
ndim = len(x0)
cov = np.diag(np.ones(ndim) * 0.01**2) # helps to tune MCMC proposal distribution

pars = pta.param_names
idx_orf_params0 = [list(pars).index(pp) for pp in pars if 'crn' in pp]
groups = [list(np.arange(0, ndim))]
[groups.append(idx_orf_params0) for ii in range(5)]

## Initiate the sampler
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, groups = groups,
                 outDir=outdir, 
                 resume=False)

jp = samp.JumpProposal(pta)
sampler.addProposalToCycle(jp.draw_from_prior, 15)
sampler.addProposalToCycle(jp.draw_from_red_prior, 15)

## Number of MCMC steps
N = int(1e5)

In [ ]:
#sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

## <span style="color:#7cfc00">Post-proccessing the output of MCMC

In [ ]:
# chain = np.loadtxt(outdir + '/chain_1.txt')
# burn = int(0.25 * chain.shape[0])
# chain = chain[burn:, :]
# chain.shape

In [ ]:
# for p, d in zip(pta.param_names, chain[:, :-4].T):
#     plt.hist(d, **hist_settings)
#     plt.title(p)
#     plt.xlabel('Paramter Value')
#     plt.ylabel('Count')
#     plt.show()

# <span style="color:#7cfc00">Detection Practice Outside ENTERPRISE

## <span style="color:#7cfc00"> Simulate a Simple PTA Data Set

In [ ]:
Npulsars = 5        ##number of pulsars
observ_time = 20    ##years
start_time = 5300   ##mjd
cadence = 14        ##days
Nrea = 10           ##number of realizations
toas = np.array([np.arange(start_time, start_time + observ_time * 365.25, cadence) for _ in range(Npulsars)])

wn_sig = 1e-7       ##white_noise level in seconds
gwb_amplitude = 2e-15
gwb_alpha = -2/3

In [ ]:
lam, bet, pnames, locs = sim.UniformPulsarDist(Numpulsars=Npulsars, name =True)

In [ ]:
s = sim.PTASIM(MG = 'TT', 
                    psrlist = pnames,
                    toas = toas,
                    psr_locs = locs.T, 
                    Amp = np.array([gwb_amplitude]),
                    alpha = np.array([gwb_alpha]), 
                    wn_sig = wn_sig,
                    Nrea = Nrea,
                    seed = 15645789)

In [ ]:
gw_res = s.total_res()

In [ ]:
pidx = 3
rea = 2

plt.plot(toas[pidx], gw_res[rea][pidx])
plt.title(pnames[pidx])
plt.xlabel('TOAS [MJD]')
plt.ylabel('Residuals [s]')
plt.show()

## <span style="color:#7cfc00">Single-pulsar Ananlysis Using Gibbs Sampling

In [ ]:
crn_bins = 5
Gsamples = int(1e4)

In [ ]:
BP = nc.BayesPower(nc.Nimapta(gw_res[rea][pidx], 
                        toas = toas[pidx], 
                        fit = False, 
                        psr_pos = locs[:, pidx],
                        wn_sigma = wn_sig),

                            red_amp = None,
                            red_gamma = None,
                            crn_bins = crn_bins,
                            Baseline = observ_time,
                            inj_amp = gwb_amplitude,
                            gamma = -1 * (2 * gwb_alpha - 3),
                            num_samples = Gsamples)

In [ ]:
_, rho = BP.gibbs_sampler(progress = True)

In [ ]:
labs = [r'$\rho_{{{0}}}$'.format(_) for _ in range(1, crn_bins + 1)]
labs

## <span style="color:#7cfc00">Results

In [ ]:
corner.corner(rho.T, color='gold', bins=20, hist_bin_factor=2, hist_kwargs={'density': True, 'lw':2}, 
              contour_kwargs={'linewidths':2}, labels = labs, quantile = True,show_titles = True,
              truth_color = 'white', desity = True, truths = BP.truth())
plt.show()

Make sure you know why the above plot looks how it looks!